In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/

Mounted at /content/drive
/content/drive/My Drive


In [ ]:
# specify imports to be used in the notebook
import tensorflow as tf
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import numpy as np
import PIL
import pathlib
import cv2

from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
CATEGORIES = ['Black-grass',
          'Charlock',
          'Cleavers',
          'Common Chickweed',
          'Common wheat',
          'Fat Hen',
          'Loose Silky-bent',
          'Maize',
          'Scentless Mayweed',
          'Shepherds Purse',
          'Small-flowered Cranesbill',
          'Sugar beet']

IMG_SIZE=255

DATADIR  = 'plant_seed/train'
DATADIR_TEST  = 'plant_seed/test'

In [ ]:
training_data=[]

def create_training_data():
    for category in CATEGORIES:

        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                
                # Pre-processing (via function pre_process) ##
                
#                 image = pre_process(new_array)
                image = new_array
                training_data.append([image,class_num])
            except Exception as e:
                print(e)
                pass
            
create_training_data()  

In [ ]:
import numpy as np

X=[]
y=[]

for categories, label in training_data:
    X.append(categories)
    y.append(label)
# X = np.array(X).reshape(train_len,-1)

y = np.array(y)

y.shape

(4750,)

In [ ]:
from keras.utils import np_utils

y = np_utils.to_categorical(y, num_classes=len(CATEGORIES))
# y_test = np_utils.to_categorical(y_test, num_classes=len(CATEGORIES))

X = np.array(X).astype('float32')

In [ ]:
print(X.shape)

print(y.shape)

(4750, 255, 255, 3)
(4750, 12)


In [ ]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

X, y = unison_shuffled_copies(X, y)

In [ ]:
# define appropriate callbacks
def training_callbacks():
    
    # save best model regularly
    save_best_model = tf.keras.callbacks.ModelCheckpoint(filepath = 'ensemble_best.h5',
        monitor = 'val_accuracy', save_best_only = True, verbose = 1)
    
    # reduce learning rate when it stops decreasing
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.4,
                              patience = 3, min_lr = 1e-10, verbose = 1, cooldown = 1)
    
    # stop training early if no further improvement
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_accuracy', min_delta = 1e-2, patience = 10, verbose = 1,
        mode = 'max', baseline = None, restore_best_weights = True
    )

    return [save_best_model, reduce_lr, early_stopping]

In [ ]:
def load_all_models():
    all_models = []
    model_names = ['efficientnet_best.h5', 'inception_best.h5', 'xception_best.h5']
    for model_name in model_names:
        filename = os.path.join('plant-seedlings-classification', model_name)
        model = tf.keras.models.load_model(filename)
        all_models.append(model)
        print('loaded:', filename)
    return all_models
models = load_all_models()
for i, model in enumerate(models):
    for layer in model.layers:
        layer.trainable = False

loaded: plant-seedlings-classification/efficientnet.h5
loaded: plant-seedlings-classification/inceptionv2.h5
loaded: plant-seedlings-classification/xception.h5


In [ ]:
count = 1
for model in models:
    for layer in model.layers:
        layer._name = layer.name + str(count)
        count = count + 1

In [ ]:
ensemble_visible = [model.input for model in models]
ensemble_outputs = [model.output for model in models]
merge = tf.keras.layers.concatenate(ensemble_outputs)
dense_1 = tf.keras.layers.Dense(10, activation = 'relu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(merge)
output = tf.keras.layers.Dense(12, activation = "softmax", activity_regularizer=tf.keras.regularizers.l2(1e-5))(dense_1)
ensemble = tf.keras.models.Model(inputs=ensemble_visible, outputs=output)

ensemble.compile(loss="categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), metrics = ["accuracy"])

In [ ]:
batch_size = 16
epochs = 100

X = [X for _ in range(len(model.input))]

history = ensemble.fit(X, y,
                    batch_size=batch_size,
                    epochs=epochs, 
                    callbacks = training_callbacks(),
                    validation_split=0.1)

In [ ]:
ensemble.save('ensemble.h5')

In [ ]:
ensemble_model = tf.keras.models.load_model('ensemble_best.h5')

In [ ]:
testing_data =[]
def create_test_data():

    for img in os.listdir(DATADIR_TEST):
        try:
            img_array=cv2.imread(os.path.join(DATADIR_TEST,img))
            new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))

            ## Pre-processing (via function pre_process)

#             image = pre_process(new_array)
            image = new_array
            testing_data.append([image,img])
        except Exception as e:
            print(e)
            pass
create_test_data()

In [ ]:
import numpy as np

X_test=[]
y_test=[]

for categories, label in testing_data:
    X_test.append(categories)
    y_test.append(label)
# X = np.array(X).reshape(train_len,-1)

y_test = np.array(y_test)

y_test.shape

(794,)

In [ ]:
ensemble_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image_input1 (InputLayer)      [(None, 255, 255, 3  0           []                               
                                )]                                                                
                                                                                                  
 image_input11 (InputLayer)     [(None, 255, 255, 3  0           []                               
                                )]                                                                
                                                                                                  
 image_input21 (InputLayer)     [(None, 255, 255, 3  0           []                               
                                )]                                                            

In [ ]:
X_tests = [np.array(X_test) for _ in range(3)]

model_preds = ensemble_model.predict(X_tests)

In [ ]:
classes = []

for data in range(0, model_preds.shape[0]):
    pred_index = model_preds[data, :].argmax(axis = -1)
    classes += [CATEGORIES[pred_index]]

In [ ]:
# generate submission csv
output_predictions = pd.DataFrame()
output_predictions['file'] = y_test
output_predictions['file'] = output_predictions['file'].str.replace(r'test/', '')
output_predictions['species'] = classes
output_predictions.to_csv('output_ensemble.csv', index = False)